In [23]:
# AUTOMORPHISM GENERATORS FOR HYPERGRAPH
# Script to compute the automorphism group of a hypergraph based on its incidence matrix
# Note: This script runs on SageMath. Use https://sagecell.sagemath.org/ for execution.

from sage.combinat.designs.incidence_structures import IncidenceStructure
from sage.groups.perm_gps.permgroup_named import SymmetricGroup
import time

# Function to convert a binary incidence matrix to a list of blocks (edges)
def binary_matrix_to_blocks(matrix):
    print("Converting binary matrix to blocks...")
    blocks = []
    for row in matrix:
        block = [i for i, x in enumerate(row) if x == 1]
        blocks.append(block)
    return blocks

# Function to extract active vertices (vertices that appear in at least one edge)
def get_active_vertices(incidence_matrix):
    active_vertices = set()
    for row in incidence_matrix:
        for idx, val in enumerate(row):
            if val == 1:
                active_vertices.add(idx)
    return sorted(active_vertices)

# Compute the automorphism group of the hypergraph
def hyp_automorphism(incidence_matrix):
    print("Computing the automorphism group...")
    active_vertices = get_active_vertices(incidence_matrix)
    print("Active vertices:", active_vertices)
    
    # Filter blocks to include only those with active vertices
    points = list(active_vertices)
    blocks = binary_matrix_to_blocks(incidence_matrix)
    filtered_blocks = [block for block in blocks if any(v in active_vertices for v in block)]
    
    print("Filtered blocks computed:", filtered_blocks)
    incidence_structure = IncidenceStructure(points, filtered_blocks)
    print("Incidence structure created.")
    
    aut_g = incidence_structure.automorphism_group()
    print("Automorphism group calculated.")
    return aut_g

# Generate symmetric group generators based on the automorphism group
def S_n_generators(aut_g, num_vertices):
    print("Generating symmetric group generators...")
    generators = []
    for perm in aut_g.gens():
        cycles = perm.cycle_tuples()
        valid_cycles = [
            tuple(v + 1 for v in cycle if v < num_vertices) 
            for cycle in cycles
            if all(v < num_vertices for v in cycle)
        ]
        if valid_cycles:
            generators.append(valid_cycles)
    return generators

# Calculate vertex degrees (orders) in the hypergraph
def vertex_orders(incidence_matrix):
    print("Calculating vertex orders...")
    vertex_order_dict = {}
    for j in range(len(incidence_matrix[0])):
        order = sum(row[j] for row in incidence_matrix)
        if order > 0:  # Include only active vertices
            vertex_order_dict[j + 1] = order  # Start vertex numbering from 1
    return vertex_order_dict

# Identify sets of vertices involved in permutation cycles
def get_vertex_permutation_sets(aut_g, num_vertices):
    print("Calculating permutation sets of vertices...")
    orbits = aut_g.orbits()
    permutation_sets = [tuple(sorted([v + 1 for v in orbit if v < num_vertices])) for orbit in orbits if len(orbit) > 1]
    return permutation_sets

print("Starting script...")

# Define the hypergraph incidence matrix here
incidence_matrix = [
    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
    [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1],
    [1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0],
    [1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0],
    [0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1],
    [0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0],
    [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
]

print("Defined incidence matrix.")

start_time = time.time()
automorphisms = hyp_automorphism(incidence_matrix)  # Compute automorphism group
end_time = time.time()

print(f"Automorphism group computation took {end_time - start_time:.2f} seconds.")

if automorphisms is not None:
    if automorphisms.order() == 1:
        print("The automorphism group is trivial.")
    else:
        print("Automorphism group is non-trivial.")
        print("Automorphism group generators are:")
        symmetric_generators = S_n_generators(automorphisms, len(incidence_matrix[0]))
        
        # Print permutation sets of vertices
        vertex_permutation_sets = get_vertex_permutation_sets(automorphisms, len(incidence_matrix[0]))
        print("Permutation sets of vertices:")
        for perm_set in vertex_permutation_sets:
            print(perm_set)

        # Print vertex orders
        vertex_order_dict = vertex_orders(incidence_matrix)
        print("Vertex orders:")
        for vertex, order in vertex_order_dict.items():
            print(f"Vertex {vertex} has degree {order}")

        # GAP Group Analysis for Automorphism Structure
        from sage.interfaces.gap import gap
        gap.eval('LoadPackage("sonata");')
        gap.eval('LoadPackage("grape");')

        def group_structure_description(generators, expected_cardinality):
            try:
                # Convert Python generators to GAP format
                generator_strings = [
                    "(" + ")(".join(",".join(map(str, cycle)) for cycle in generator) + ")"
                    for generator in generators
                ]
                generators_combined = ", ".join(generator_strings)
                gap_command = f"gap_group := Group({generators_combined});"
                gap.eval(gap_command)

                # Retrieve group properties
                actual_cardinality = gap.eval('Size(gap_group);')
                structure_description = gap.eval('StructureDescription(gap_group);')

                if int(actual_cardinality) != expected_cardinality:
                    print(f"Warning: Cardinality mismatch (Expected: {expected_cardinality}, Found: {actual_cardinality})")
                else:
                    print(f"Cardinality matches expected value of {expected_cardinality}")

                minimal_generators = gap.eval('SmallGeneratingSet(gap_group);')
                print("Small Generating Set:")
                print(minimal_generators)

                return structure_description

            except RuntimeError as e:
                print(f"Error during group analysis: {e}")
                return None

        group_structure = group_structure_description(symmetric_generators, automorphisms.cardinality())
        if group_structure:
            print(f"Group Structure: {group_structure}")

else:
    print("Automorphism group could not be computed.")

print("Script finished.")


Starting script...
Defined incidence matrix.
Computing the automorphism group...
Active vertices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Converting binary matrix to blocks...
Filtered blocks computed: [[0, 2, 4, 6, 8, 10], [0, 2, 4, 6, 9, 11], [0, 2, 5, 7, 8, 10], [0, 2, 5, 7, 9, 11], [1, 3, 4, 6, 8, 10], [1, 3, 4, 6, 9, 11], [1, 3, 5, 7, 8, 10], [1, 3, 5, 7, 9, 11]]
Incidence structure created.
Automorphism group calculated.
Automorphism group computation took 0.00 seconds.
Automorphism group is non-trivial.
Automorphism group generators are:
Generating symmetric group generators...
Calculating permutation sets of vertices...
Permutation sets of vertices:
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
Calculating vertex orders...
Vertex orders:
Vertex 1 has degree 4
Vertex 2 has degree 4
Vertex 3 has degree 4
Vertex 4 has degree 4
Vertex 5 has degree 4
Vertex 6 has degree 4
Vertex 7 has degree 4
Vertex 8 has degree 4
Vertex 9 has degree 4
Vertex 10 has degree 4
Vertex 11 has degree 4
Vertex 

In [24]:
# RETURNS THE PRIMAL GRAPH, CONFORMALITY, AND NORMALITY

# Import necessary SageMath libraries for graph construction and matrix operations
from sage.graphs.graph import Graph  # Library for working with graphs
from sage.matrix.constructor import Matrix  # Library for matrix construction
from sage.rings.integer_ring import ZZ  # Library for working with integer elements in SageMath

# Define the incidence matrix of the hypergraph H
# Each row represents a hyperedge, and each column corresponds to a vertex.
H = Matrix(ZZ, 

           
[[0, 0, 0, 0, 1, 0, 0, 1],  
 [0, 0, 0, 0, 1, 0, 1, 0]]  

           
)



# Function to compute the primal graph of a hypergraph from its incidence matrix
def primal_graph(H):
    """
    Compute the primal graph of a hypergraph.
    
    Args:
    H (Matrix): Incidence matrix of the hypergraph.

    Returns:
    dict: A dictionary containing vertices and edges of the primal graph.
    """
    # Initialize the primal graph G
    n_rows, n_cols = H.nrows(), H.ncols()  # Get the dimensions of the incidence matrix
    G = Graph()  # Create an empty graph

    # Add vertices to the graph (one for each column in the incidence matrix)
    for i in range(n_cols):
        G.add_vertex(i)
    
    # Add edges based on the incidence matrix
    # Two vertices are connected if they belong to the same hyperedge
    for row in H.rows():
        hyperedge = [i for i in range(len(row)) if row[i] == 1]  # Extract vertices in the hyperedge
        G.add_edges([(hyperedge[i], hyperedge[j]) 
                     for i in range(len(hyperedge)) 
                     for j in range(i + 1, len(hyperedge))])  # Add edges for all pairs in the hyperedge
    
    # Return vertices and edges of the primal graph in a dictionary format
    vertices = list(G.vertices())
    edges = list(G.edges(labels=False))  # Edges without weights or labels
    
    return {
        'vertices': vertices,
        'edges': edges
    }

# Compute the primal graph from the incidence matrix
result = primal_graph(H)
edges = result['edges']  # Extract the edges of the primal graph

# Print the vertices and edges of the primal graph
for key, value in result.items():
    print(f"{key} = {value}")

# Construct the graph G using the edges computed from the incidence matrix
G = Graph(edges)

# Conformality Test
# A hypergraph is conformal if every maximal clique of the primal graph is contained in at least one hyperedge.
maximal_cliques = G.cliques_maximal()  # Extract all maximal cliques of the primal graph
H_hyperedges = [set([i for i in range(len(row)) if row[i] == 1]) for row in H.rows()]  # Convert hyperedges to sets
non_conformal_cliques = [set(clique) for clique in maximal_cliques 
                         if not any(set(clique).issubset(hyperedge) for hyperedge in H_hyperedges)]  # Find non-conformal cliques
is_conformal = len(non_conformal_cliques) == 0  # Check if all maximal cliques are conformal

# Normality Test
# A hypergraph is normal if every clique of the primal graph is contained in at least one hyperedge.
all_cliques = G.all_cliques()  # Extract all cliques of the primal graph
non_normal_cliques = [set(clique) for clique in all_cliques 
                      if not any(set(clique).issubset(hyperedge) for hyperedge in H_hyperedges)]  # Find non-normal cliques
is_normal = len(non_normal_cliques) == 0  # Check if all cliques are normal

# Output the result of the conformality test
if is_conformal:
    print("The hypergraph is conformal.")
else:
    print(f"The hypergraph is not conformal. Non-conformal cliques: {non_conformal_cliques}")

# Output the result of the normality test
if is_normal:
    print("The hypergraph is normal.")
else:
    print(f"The hypergraph is not normal. Non-normal cliques: {non_normal_cliques}")


vertices = [0, 1, 2, 3, 4, 5, 6, 7]
edges = [(4, 6), (4, 7)]
The hypergraph is conformal.
The hypergraph is normal.


In [657]:
# Import necessary libraries from SageMath
from sage.graphs.graph import Graph
from itertools import combinations
from sage.graphs.graph_decompositions.rankwidth import rank_decomposition

# Define the edges of the graph
# Input graph defined by its edge list
edges = [(2, 6), (2, 7), (4, 5), (5, 6), (5, 7), (6, 7)]

# Create a graph G from the edge list
G = Graph(edges)

# Check various properties of the graph
is_cograph = G.is_cograph()  # Check if the graph is a cograph (P4-free graph)
is_perfect = G.is_perfect()  # Check if the graph is perfect
is_split = G.is_split()  # Check if the graph is a split graph
is_weakly_chordal = G.is_weakly_chordal()  # Check if the graph is weakly chordal
is_triangle_free = G.is_triangle_free()  # Check if the graph is triangle-free
is_bipartite = G.is_bipartite()  # Check if the graph is bipartite
is_chordal = G.is_chordal()  # Check if the graph is chordal
is_chordal_bipartite = is_weakly_chordal and is_bipartite  # Check if the graph is chordal bipartite

# Function to check if the graph contains an induced P4 (path of length 4)
def has_induced_P4(G):
    for vertices in combinations(G.vertices(), 4):  # Iterate over all combinations of 4 vertices
        subgraph = G.subgraph(vertices)  # Get the subgraph induced by the 4 vertices
        # Check if the subgraph is isomorphic to a P4 (path graph with 4 vertices)
        if subgraph.is_isomorphic(Graph([(0, 1), (1, 2), (2, 3)])):
            return "Graph has P4"  # Return if a P4 is found
    return "Graph does not have P4"  # Return if no P4 is found

# Compute the rank width of the graph
# Distance hereditary graphs have their rank-width at most 1. 
rank_width = rank_decomposition(G)

# Output the graph properties
print(f"The underlying primal graph is a cograph: {is_cograph}")
print(f"The underlying primal graph is perfect: {is_perfect}")
print(f"The underlying primal graph is a split graph: {is_split}")
print(f"The underlying primal graph is weakly chordal: {is_weakly_chordal}")
print(f"The underlying primal graph is triangle-free: {is_triangle_free}")
print(f"The underlying primal graph is bipartite: {is_bipartite}")
print(f"The underlying primal graph is chordal: {is_chordal}")
print(f"The underlying primal graph is chordal bipartite: {is_chordal_bipartite}")
print(f"Rank Width of G is: {rank_width}")

# Output whether the graph contains an induced P4
print(has_induced_P4(G))


The underlying primal graph is a cograph: False
The underlying primal graph is perfect: True
The underlying primal graph is a split graph: True
The underlying primal graph is weakly chordal: True
The underlying primal graph is triangle-free: False
The underlying primal graph is bipartite: False
The underlying primal graph is chordal: True
The underlying primal graph is chordal bipartite: False
Rank Width of G is: (1, Graph on 9 vertices)
Graph has P4


In [25]:
# Import the GAP interface from SageMath
gap = sage.interfaces.gap.gap


# D4: Dihedral group of order 8, 
gap.eval("D4 := DihedralGroup(8);")
# S2: Symmetric group of order 2, 
gap.eval("S2 := SymmetricGroup(2);")

# Construct the wreath product of D4 and S2
# The wreath product combines the two groups based on a specific permutation structure
gap.eval("WreathProductD4S3 := WreathProduct(D4, S2);")

# Retrieve and print the order of the wreath product
# The order of a wreath product is calculated as |G|^|H| × |H| for groups G and H
order = gap.eval("Size(WreathProductD4S3);")
print(f"Order of the wreath product D4 wr S2: {order}")


try:
    structure = gap.eval("StructureDescription(WreathProductD4S3);")
    print(f"Structure of the wreath product D4 wr S2: {structure}")
except RuntimeError:
    print("Structure description not available.")




Order of the wreath product D4 wr S2: 128
Structure of the wreath product D4 wr S2: "(((C2 x C2 x C2 x C2) : C2) : C2) : C2"


In [16]:
"((((C2 x C2 x ((C2 x C2 x C2 x C2) : C2)) : C2) : C3) : C2) : C2"


Error in evaluating groups: error evaluating "G1 := DirectProduct(CyclicGroup(3), CyclicGroup(3), CyclicGroup(3), CyclicGroup(3));     G2 := DirectProduct(CyclicGroup(2), CyclicGroup(2), CyclicGroup(2), CyclicGroup(2));     phi1 := HomomorphismByFunction(G2, G2, x -> x, x -> x);       H1 := SemidirectProduct(G2, Image(phi1));     phi2 := HomomorphismByFunction(H1, CyclicGroup(2), x -> x, x -> x);     H2 := SemidirectProduct(H1, Image(phi2));     phi3 := HomomorphismByFunction(H2, CyclicGroup(2), x -> x, x -> x);     Group1 := SemidirectProduct(H2, Image(phi3));":
argument of type 'RuntimeError' is not iterable
Are the groups isomorphic? None


In [19]:
# Define the local symmetry group (Dihedral Group D4 for each square)
D4 = DihedralGroup(8);

# Define the global permutation group (Symmetric Group S3 for permuting the squares)
S3 = SymmetricGroup(3);

# Define the wreath product D4 wr S3
Wreath = WreathProduct(D4, S3);

# Define the action of the wreath product on 12 vertices (3 disjoint squares)
ActionOnVertices = Action(Wreath, [1..12]);

# Verify the order of the wreath product
SizeWreath = Size(Wreath);
Print("Order of the wreath product D4 wr S3: ", SizeWreath, "\n");

# Example: Check the action of a specific generator on a vertex
ImageOfGenerator = Image(ActionOnVertices, Wreath.1, 1);
Print("Image of vertex 1 under the action of the first generator: ", ImageOfGenerator, "\n");


NameError: name 'WreathProduct' is not defined